In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

# Testing Suite

### Dependencies

In [1]:
import chess
import json
import pprint
import numpy as np
from typing import Callable

import import_ipynb
import Game
from Globals import Globals
from MinimaxAlgorithm import make_move_minimax, make_move_minimax_memoization
from AlphaBetaAlgorithm import make_move_alphabeta, \
        make_move_alphabeta_progressive_deepening

importing Jupyter notebook from Game.ipynb


importing Jupyter notebook from Globals.ipynb


importing Jupyter notebook from MinimaxAlgorithm.ipynb


importing Jupyter notebook from Util.ipynb


importing Jupyter notebook from AlphaBetaAlgorithm.ipynb


In general, a game of chess can be initialized through an instance of the Game class. To be precise, the creation of the Game object can be configured with the following parameters:

``make_move_algorithm_white (Callable, default: None):`` 
The function that the AI controlling the white pieces should use to make a new move, or None if the white pieces are player-controlled.

``make_move_algorithm_black (Callable, default: None):``  
The function that the AI controlling the black pieces should use to make a new move, or None if the black pieces are player-controlled.

``search_depth_white (int, default: 3):``  
The search depth for the white AI's search algorithm.

``search_depth_black (int, default: 3):``  
The search depth for the black AI's search algorithm.

``opening_book (str, default: 'Resources/baron30.bin'):``  
The path to the desired opening book the AIs will use, or None if no opening book is desired.

``endgame_tablebase_dir (str, default: 'Resources/Gaviota'):``  
The path to the directory of the desired endgame tablebase the AIs will use, or None if no endgame tablebase is desired.

The possible algorithms are:  

1. make_random_move
2. make_move_minimax
3. make_move_minimax_memoization
4. make_move_alphabeta
5. make_move_alphabeta_progressive_deepening  

Furthermore, it is recommended not to set the opening_book and endgame_tablebase_dir arguments to None in order to boost efficiency and reduce the time needed by the AI to make a move, unless the test case requires that the algorithms are tested without these aids.

This test shows a basic game which can be played. The AIs are represented by the minimax search algorithm and the alpha-beta pruning algorithm.  
In the following case, a chess problem is given in the function call. This test is written simply to test the basic functionality of the search algorithms as well as the initialization of a chess problem.

In [ ]:
chess_problem_test_game = Game.Game(make_move_minimax,
        make_move_alphabeta, endgame_tablebase_dir=None)
chess_problem_test_game.play(chess_problem='EK')

In the following, a normal chess game is initialized and played using the minimax search and alpha-beta pruning algorithms. This is a more complex tests as a whole game of chess is played rather than only a partial game, as was the case in the chess problem test above.

In [ ]:
normal_game = Game.Game(
    make_move_algorithm_white=make_move_minimax, 
    make_move_algorithm_black=make_move_alphabeta
)

normal_game.play()

In [ ]:
test_game = Game.Game(
    make_move_algorithm_white=make_move_alphabeta,
    search_depth_white=5,
    make_move_algorithm_black=make_move_alphabeta_progressive_deepening,
    search_depth_black=5
)

test_game.play()

In [ ]:
white_times_mean = np.mean(test_game.move_times[chess.WHITE])
white_times_std = np.std(test_game.move_times[chess.WHITE])
black_times_mean = np.mean(test_game.move_times[chess.BLACK])
black_times_std = np.std(test_game.move_times[chess.BLACK])
print(f'AlphaBeta (white) move times. Mean: {white_times_mean}, ' \
        + f'std. dev: {white_times_std}')
print(f'AlphaBeta Prog.Deep. (black) move times. Mean: {black_times_mean}, ' \
        + f'std. dev: {black_times_std}')

___

#### perform_chess_problem_tests
Runs the alpha-beta pruning algorithm against all available chess problems in order to test the functionality and effeciveness of this algorithm.

###### __<u>Arguments</u>__
``use_endgame_library (bool):``  
Whether or not the endgame library can be used in the tests.  

``search_depth_auto (bool):``  
Whether or not the maximum search depth of the alpha-beta pruning algorithm should be set to the minimum amount of moves the chess problem takes to solve. In theory, setting this to True should always yield an optimal solution.

###### __<u>Side effects</u>__
- The current visual output is overwritten.
- All defined tests are run.
- Test playthrough information is tracked and printed.

In [ ]:
def perform_chess_problem_tests(
    use_endgame_library: bool,
    search_depth_auto: bool
) -> None:

    if use_endgame_library:
        test_game = Game.Game(make_move_alphabeta, make_move_alphabeta)
    else:
        test_game = Game.Game(make_move_alphabeta, make_move_alphabeta, 
                endgame_tablebase_dir=None)

    for c in Globals.CHESS_PROBLEMS:
        test_game.play(chess_problem=c, search_depth_auto=search_depth_auto,
                automation=True)

    pprint.pprint(test_game.problem_playthroughs)
    print(f'\n{test_game.wins}')

First, we test the search algorithm without an endgame tablebase. The search depth is the default value (3), therefore more complex chess problems are unlikely to be solved in the fewest possible moves.


In [ ]:
perform_chess_problem_tests(use_endgame_library=False, search_depth_auto=False)

Next, we test the search algorithm without an endgame tablebase, but with a search depth that's theoretically enough to solve the chess problem perfectly. This is simply the amount of moves that the chess problem takes to solve at minimum, because looking that many moves into the future should tell the algorithm how to reach a checkmate scenario.


In [ ]:
perform_chess_problem_tests(use_endgame_library=False, search_depth_auto=True)

Lastly, we test the search algorithm with an endgame tablebase, in order to test if this tablebase is incorporated into the search function correctly. The search depth is set to the minimum move count to solve the problem, in order to see if the endgame tablebase is triggered at any point during an "intended" solution.

In [ ]:
perform_chess_problem_tests(use_endgame_library=True, search_depth_auto=True)

#### perform_full_game_test
This is a full game test. Therefore a full game of chess is played by two AIs instead of a single chess problem. The game is defined through the arguments below.

###### __<u>Arguments</u>__
``make_move_white (Callable):``  
The function that the AI controlling the white pieces should use to make a new move.  

``algo_name_white (str):``  
The name of the algorithm that the white AI uses.  

``make_move_black (Callable):``  
The function that the AI controlling the black pieces should use to make a new move.  

``algo_name_black (str):``  
The name of the algorithm that the black AI uses.  

``use_opening_book (bool):``  
Whether or not an opening book should be used in the test.  

``use_endgame_tablebase (bool):``  
Whether or not an endgame tablebase should be used in the test.  

###### __<u>Side effects</u>__
- The current visual output is overwritten.
- The defined game of chess is played.
- Test information is tracked and printed.
- Information about the move times is calculated and displayed.

###### __<u>Returns _(Game.Game)_</u>__
The Game object of the game of chess that has been played.

In [ ]:
def perform_full_game_test(
    make_move_white: Callable,
    algo_name_white: str,
    make_move_black: Callable,
    algo_name_black: str,
    use_opening_book: bool,
    use_endgame_tablebase: bool
) -> Game.Game:

    params = {}
    if not use_opening_book: params['opening_book'] = None
    if not use_endgame_tablebase: params['endgame_tablebase_dir'] = None
    game = Game.Game(make_move_white, make_move_black, **params)

    game.play()

    white_times_mean = np.mean(game.move_times[chess.WHITE])
    white_times_std = np.std(game.move_times[chess.WHITE])
    black_times_mean = np.mean(game.move_times[chess.BLACK])
    black_times_std = np.std(game.move_times[chess.BLACK])
    print(f'{algo_name_white} (white) move times. Mean: {white_times_mean}, ' \
            + f'std. dev: {white_times_std}')
    print(f'{algo_name_black} (black) move times. Mean: {black_times_mean}, ' \
            + f'std. dev: {black_times_std}')

---
In the following, every AI plays against every AI (but not against itself). Each combination is tested with only an opening book, only an endgame tablebase and neither. This is done to retrieve various pieces of useful information:
- If the algorithms function as intended. If a test ends in an unpredicted manner, it may be a sign that at least one of the involved algorithms is defective.
- Which algorithm can make the most effective moves. In theory, every search algorithm should be equally effective, whereas the random algorithm is inferior.
- Which algorithm can find moves the quickest. This is only applicable to search algorithms. In theory, alpha-beta pruning should be faster than minimax search. 
- How effective memoization is in a minimax search (and by extension in other searches).
- How much progressive deepening speeds up alpha-beta pruning.
- How effective opening books are at speeding up search algorithms.
- How effective endgame tablebases are at speeding up search algorithms.

First, the different tests are defined.  After that, each cell contains one test, so the output for each test is not overwritten by the output of the following test.

___

In [ ]:
game_minimax_minimemo = perform_full_game_test(
    make_move_white=make_move_minimax,
    algo_name_white='Minimax',
    make_move_black=make_move_minimax_memoization,
    algo_name_black='Minimax with memoization',
    use_opening_book=False,
    use_endgame_tablebase=False
)

In [ ]:
game_minimax_minimemo_opening = perform_full_game_test(
    make_move_white=make_move_minimax,
    algo_name_white='Minimax',
    make_move_black=make_move_minimax_memoization,
    algo_name_black='Minimax with memoization',
    use_opening_book=True,
    use_endgame_tablebase=False
)

In [ ]:
game_minimax_minimemo_endgame = perform_full_game_test(
    make_move_white=make_move_minimax,
    algo_name_white='Minimax',
    make_move_black=make_move_minimax_memoization,
    algo_name_black='Minimax with memoization',
    use_opening_book=False,
    use_endgame_tablebase=True
)

___

In [ ]:
game_minimax_alphabeta = perform_full_game_test(
    make_move_white=make_move_minimax,
    algo_name_white='Minimax',
    make_move_black=make_move_alphabeta,
    algo_name_black='Alpha-beta',
    use_opening_book=False,
    use_endgame_tablebase=False
)

In [ ]:
game_minimax_alphabeta_opening = perform_full_game_test(
    make_move_white=make_move_minimax,
    algo_name_white='Minimax',
    make_move_black=make_move_alphabeta,
    algo_name_black='Alpha-beta',
    use_opening_book=True,
    use_endgame_tablebase=False
)

In [ ]:
game_minimax_alphabeta_endgame = perform_full_game_test(
    make_move_white=make_move_minimax,
    algo_name_white='Minimax',
    make_move_black=make_move_alphabeta,
    algo_name_black='Alpha-beta',
    use_opening_book=False,
    use_endgame_tablebase=True
)

___

In [ ]:
game_minimemo_alphabeta = perform_full_game_test(
    make_move_white=make_move_minimax_memoization,
    algo_name_white='Minimax with memoization',
    make_move_black=make_move_alphabeta,
    algo_name_black='Alpha-beta',
    use_opening_book=False,
    use_endgame_tablebase=False
)

In [ ]:
game_minimemo_alphabeta_opening = perform_full_game_test(
    make_move_white=make_move_minimax_memoization,
    algo_name_white='Minimax with memoization',
    make_move_black=make_move_alphabeta,
    algo_name_black='Alpha-beta',
    use_opening_book=True,
    use_endgame_tablebase=False
)

In [ ]:
game_minimemo_alphabeta_endgame = perform_full_game_test(
    make_move_white=make_move_minimax_memoization,
    algo_name_white='Minimax with memoization',
    make_move_black=make_move_alphabeta,
    algo_name_black='Alpha-beta',
    use_opening_book=False,
    use_endgame_tablebase=True
)

___

In [ ]:
game_alphabeta_pd = perform_full_game_test(
    make_move_white=make_move_alphabeta,
    algo_name_white='Alpha-beta',
    make_move_black=make_move_alphabeta_progressive_deepening,
    algo_name_black='Alpha-beta progressive deepening',
    use_opening_book=False,
    use_endgame_tablebase=False
)

In [ ]:
game_alphabeta_pd_opening = perform_full_game_test(
    make_move_white=make_move_alphabeta,
    algo_name_white='Alpha-beta',
    make_move_black=make_move_alphabeta_progressive_deepening,
    algo_name_black='Alpha-beta progressive deepening',
    use_opening_book=True,
    use_endgame_tablebase=False
)

In [ ]:
game_alphabeta_pd_endgame = perform_full_game_test(
    make_move_white=make_move_alphabeta,
    algo_name_white='Alpha-beta',
    make_move_black=make_move_alphabeta_progressive_deepening,
    algo_name_black='Alpha-beta progressive deepening',
    use_opening_book=False,
    use_endgame_tablebase=True
)

___

#### perform_fen_test
Makes a move for both the alpha-beta pruning algorithm and the minimax search algorithm on a board with a given FEN code, then returns the results.

###### __<u>Arguments</u>__
``fencode (str):``  
The FEN code of the desired board.

###### __<u>Returns _(dict<str, tuple>)_</u>__
For the keys 'alphabeta' and 'minimax', a tuple containing the move and the corresponding score.

In [ ]:
def perform_fen_test(fencode: str):
    fen_test_game = Game.Game(make_move_algorithm_white=make_move_alphabeta,
            make_move_algorithm_black=make_move_alphabeta,
            opening_book=None, endgame_tablebase_dir=None)
    fen_test_game.board = chess.Board(fen=fencode)
    fen_test_game.board.ending = False
    score_alpha, move_alpha, _ = fen_test_game.make_move[chess.WHITE](
            fen_test_game.board, chess.WHITE, 3, None, True)
    fen_test_game.display()
    fen_test_game = Game.Game(make_move_algorithm_white=make_move_minimax,
            make_move_algorithm_black=make_move_alphabeta,
            opening_book=None, endgame_tablebase_dir=None)
    fen_test_game.board = chess.Board(fen=fencode)
    fen_test_game.board.ending = False
    score_minimax, move_minimax, _ = fen_test_game.make_move[chess.WHITE](
            fen_test_game.board, chess.WHITE, 3, None, True)
    fen_test_game.display()

    return { 'alphabeta': (move_alpha, score_alpha), 'minimax': (move_minimax, score_minimax) }

In the following test, we check if the two basic algorithms (minimax and alpha-beta) make the same move for a given board. Since alpha-beta pruning is simply an optimized version of minimax search, this should be the case.

In [ ]:
fen = 'r7/4Kn2/5Q2/PR6/2nbkB2/2P3P1/2p1p1Np/8 w - - 0 1'
display_game = Game.Game(make_move_algorithm_white=make_move_alphabeta,
        make_move_algorithm_black=make_move_minimax, opening_book=None,
        endgame_tablebase_dir=None)
display_game.board = chess.Board(fen=fen)
display_game.display()

In [ ]:
moves = perform_fen_test(fen)
moves